In [2]:
# Installationsbefehle (normalerweise nur einmal pro Session ausführen)
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git # Nur einmal ausführen, um das Repo zu klonen
!pip install panel --quiet
!pip install jupyter_bokeh --quiet # Bokeh ist eine Abhängigkeit von Panel für Jupyter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Behalten wir für den Fall, dass es für andere Teile gebraucht wird
import seaborn as sns # Behalten wir für den Fall
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets # Momentan nicht aktiv für F3, aber für andere Teile evtl.
from IPython.display import display, clear_output # Momentan nicht aktiv für F3
import panel as pn
pn.extension('plotly',sizing_mode="stretch_width") # sizing_mode global setzen für alle Plotly Panes

Cloning into 'BINA-Projekt'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 236 (delta 41), reused 4 (delta 2), pack-reused 165 (from 1)
Receiving objects: 100% (236/236), 5.16 MiB | 9.04 MiB/s, done.
Resolving deltas: 100% (130/130), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.3 MB/s eta 0:00:00


In [3]:
# Datei einlesen
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

# Daten überprüfen
print(df.head())

   Jahr           Rubrik Energietraeger        TJ
0  1980  Bruttoverbrauch   Elektrizität  -29450.0
1  1980  Bruttoverbrauch  Erdölprodukte  321250.0
2  1980  Bruttoverbrauch      Fernwärme       0.0
3  1980  Bruttoverbrauch            Gas   36280.0
4  1980  Bruttoverbrauch    Holzenergie   26280.0


In [4]:
# Harmlose Bokeh-Warnmeldung ausblenden
import warnings
from bokeh.util.warnings import BokehUserWarning
warnings.filterwarnings("ignore", category=BokehUserWarning)

# Grundlegende Datenaufbereitung
df['TJ'] = pd.to_numeric(df['TJ'], errors='coerce').fillna(0)
df['Jahr'] = df['Jahr'].astype(int)

# --- Definitionen für F3 ---
erneuerbare_inland_basis_f3 = ["Holzenergie", "Wasserkraft", "Uebrige erneuerbare Energien"]
muell_industrieabfaelle_f3 = ["Müll und Industrieabfälle"]
erneuerbare_inland_mit_muell_f3 = erneuerbare_inland_basis_f3 + muell_industrieabfaelle_f3
fossile_import_traeger_f3 = ["Erdölprodukte", "Gas", "Kohle", "Rohöl"]

# Farbcodierung für F3
farben_erneuerbar_inland_f3 = {
    "Holzenergie": "#1f77b4", "Wasserkraft": "#aec7e8", "Uebrige erneuerbare Energien": "#72bcd4",
    "Müll und Industrieabfälle": "#2ca02c"
}
farben_fossil_import_f3 = {
    "Erdölprodukte": "#ff7f0e", "Gas": "#ffbb78", "Kohle": "#d62728", "Rohöl": "#ff9896"
}
color_map_f3_szenario1_detail = {
    **{k: v for k, v in farben_erneuerbar_inland_f3.items() if k not in muell_industrieabfaelle_f3},
    **farben_fossil_import_f3
}
color_map_f3_szenario2_detail = {**farben_erneuerbar_inland_f3, **farben_fossil_import_f3}

# --- Hilfsfunktion zur Datenaggregation (bleibt hier, da sie von beiden Szenarien genutzt wird) ---
def prepare_f3_data(df_input, erneuerbare_liste, fossile_liste, szenario_label=""):
    df_ern_inland_detail = df_input[
        (df_input['Rubrik'] == "Inlandproduktion") &
        (df_input['Energietraeger'].isin(erneuerbare_liste))
    ].groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_ern_inland_detail['Hauptkategorie'] = f'Inländ. Erneuerbare {szenario_label}'.strip()

    ern_inland_summe = df_ern_inland_detail.groupby('Jahr')['TJ'].sum().reset_index()
    ern_inland_summe = ern_inland_summe.rename(columns={'TJ': 'Erneuerbare_Inland_TJ'})

    df_foss_import_detail = df_input[
        (df_input['Rubrik'] == "Import") &
        (df_input['Energietraeger'].isin(fossile_liste))
    ].groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_foss_import_detail['Hauptkategorie'] = 'Importe fossiler Energieträger'

    foss_import_summe = df_foss_import_detail.groupby('Jahr')['TJ'].sum().reset_index()
    foss_import_summe = foss_import_summe.rename(columns={'TJ': 'Fossile_Import_TJ'})

    df_detail_kombiniert = pd.concat([df_ern_inland_detail, df_foss_import_detail])

    df_vergleich = pd.merge(ern_inland_summe, foss_import_summe, on='Jahr', how='outer').fillna(0)
    df_vergleich = df_vergleich.sort_values(by='Jahr').reset_index(drop=True)

    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Erneuerbare_Inland_TJ'] / df_vergleich['Fossile_Import_TJ'].replace(0, np.nan)
    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'].fillna(0)

    summe_energien = df_vergleich['Erneuerbare_Inland_TJ'] + df_vergleich['Fossile_Import_TJ']
    label_suffix = f" {szenario_label}".rstrip() # Stellt sicher, dass kein Leerzeichen am Ende ist, wenn Label leer
    df_vergleich[f'Anteil_Erneuerbare_Inland_Prozent{label_suffix}'] = (df_vergleich['Erneuerbare_Inland_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)
    df_vergleich[f'Anteil_Fossile_Import_Prozent{label_suffix}'] = (df_vergleich['Fossile_Import_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)

    return df_detail_kombiniert, df_vergleich

# --- Daten für Szenarien vorbereiten ---
df_detail_s1_f3, df_vergleich_s1_f3 = prepare_f3_data(df, erneuerbare_inland_basis_f3, fossile_import_traeger_f3, szenario_label="(Basis)")
df_detail_s2_f3, df_vergleich_s2_f3 = prepare_f3_data(df, erneuerbare_inland_mit_muell_f3, fossile_import_traeger_f3, szenario_label="(inkl. Müll)")

In [5]:
def plot_f3_s1_absolut_aggregiert_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_vergleich_s1_f3['Jahr'], y=df_vergleich_s1_f3['Erneuerbare_Inland_TJ'],
                             mode='lines+markers', name='Inländ. Erneuerbare (Basis)', line=dict(color='#1f77b4')))
    fig.add_trace(go.Scatter(x=df_vergleich_s1_f3['Jahr'], y=df_vergleich_s1_f3['Fossile_Import_TJ'],
                             mode='lines+markers', name='Importe fossiler Energieträger', line=dict(color='#ff7f0e')))
    fig.update_layout(title_text="Abs. Mengen (aggregiert): Erneuerbare (Basis) vs. Fossile Importe",
                      title_x=0.5, xaxis_title="Jahr", yaxis_title="Energiemenge (TJ)", hovermode="x unified", legend_title_text='Kategorie')
    return fig

In [6]:
def plot_f3_s1_absolut_detail_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = px.area(df_detail_s1_f3, x="Jahr", y="TJ", color="Energietraeger", line_group="Hauptkategorie",
                  color_discrete_map=color_map_f3_szenario1_detail,
                  title="Abs. Mengen (detailliert): Erneuerbare (Basis) vs. Fossile Importe",
                  labels={"TJ": "Energiemenge (TJ)", "Energietraeger": "Energieträger"},
                  category_orders={"Energietraeger": erneuerbare_inland_basis_f3 + fossile_import_traeger_f3})
    fig.update_layout(title_x=0.5, legend_title_text='Energieträger', hovermode="x unified")
    return fig

In [7]:
def plot_f3_s1_anteil_100prozent_aggregiert_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    # Korrekte Spaltennamen verwenden, wie in prepare_f3_data definiert
    df_anteile_melted = df_vergleich_s1_f3.melt(id_vars=['Jahr'],
                                              value_vars=['Anteil_Erneuerbare_Inland_Prozent (Basis)', 'Anteil_Fossile_Import_Prozent (Basis)'],
                                              var_name='Kategorie_Anteil', value_name='Prozentualer_Anteil')
    df_anteile_melted['Kategorie_Anteil'] = df_anteile_melted['Kategorie_Anteil'].replace({
        'Anteil_Erneuerbare_Inland_Prozent (Basis)': 'Anteil Inländ. Erneuerbare (Basis)',
        'Anteil_Fossile_Import_Prozent (Basis)': 'Anteil Fossile Importe'
    })
    fig = px.area(df_anteile_melted, x="Jahr", y="Prozentualer_Anteil", color="Kategorie_Anteil",
                  title="Anteilige Entwicklung (aggregiert, 100% Ansicht)",
                  labels={"Prozentualer_Anteil": "Prozentualer Anteil (%)"},
                  color_discrete_map={'Anteil Inländ. Erneuerbare (Basis)': '#1f77b4', 'Anteil Fossile Importe': '#ff7f0e'},
                  groupnorm='percent')
    fig.update_layout(title_x=0.5, legend_title_text='Energiekategorie', hovermode="x unified")
    fig.update_yaxes(ticksuffix="%")
    return fig

In [8]:
def plot_f3_s1_verhaeltnis_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = px.line(df_vergleich_s1_f3, x="Jahr", y="Verhaeltnis_ErnIn_vs_FossImp",
                  title="Verhältnis Inl. Erneuerbare (Basis) zu Fossilen Importen",
                  labels={"Verhaeltnis_ErnIn_vs_FossImp": "Verhältnis"}, markers=True)
    fig.add_hline(y=1, line_dash="dash", line_color="grey", annotation_text="Gleichstand")
    fig.update_layout(title_x=0.5, hovermode="x unified")
    return fig

In [9]:
def plot_f3_s2_absolut_aggregiert_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Erneuerbare_Inland_TJ'],
                             mode='lines+markers', name='Inländ. Erneuerbare (inkl. Müll)', line=dict(color=farben_erneuerbar_inland_f3["Müll und Industrieabfälle"])))
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Fossile_Import_TJ'],
                             mode='lines+markers', name='Importe fossiler Energieträger', line=dict(color=farben_fossil_import_f3["Erdölprodukte"])))
    fig.update_layout(title_text="Abs. Mengen (aggregiert): Erneuerbare (inkl. Müll) vs. Fossile Importe",
                      title_x=0.5, xaxis_title="Jahr", yaxis_title="Energiemenge (TJ)", hovermode="x unified", legend_title_text='Kategorie')
    return fig

In [10]:
def plot_f3_s2_absolut_detail_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = px.area(df_detail_s2_f3, x="Jahr", y="TJ", color="Energietraeger", line_group="Hauptkategorie",
                  color_discrete_map=color_map_f3_szenario2_detail,
                  title="Abs. Mengen (detailliert): Erneuerbare (inkl. Müll) vs. Fossile Importe",
                  labels={"TJ": "Energiemenge (TJ)", "Energietraeger": "Energieträger"},
                  category_orders={"Energietraeger": erneuerbare_inland_mit_muell_f3 + fossile_import_traeger_f3})
    fig.update_layout(title_x=0.5, legend_title_text='Energieträger', hovermode="x unified")
    return fig

In [11]:
def plot_f3_s2_anteil_100prozent_aggregiert_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    # Korrekte Spaltennamen verwenden
    df_anteile_melted = df_vergleich_s2_f3.melt(id_vars=['Jahr'],
                                              value_vars=['Anteil_Erneuerbare_Inland_Prozent (inkl. Müll)', 'Anteil_Fossile_Import_Prozent (inkl. Müll)'],
                                              var_name='Kategorie_Anteil', value_name='Prozentualer_Anteil')
    df_anteile_melted['Kategorie_Anteil'] = df_anteile_melted['Kategorie_Anteil'].replace({
        'Anteil_Erneuerbare_Inland_Prozent (inkl. Müll)': 'Anteil Inländ. Erneuerbare (inkl. Müll)',
        'Anteil_Fossile_Import_Prozent (inkl. Müll)': 'Anteil Fossile Importe' # Name konsistent halten
    })
    fig = px.area(df_anteile_melted, x="Jahr", y="Prozentualer_Anteil", color="Kategorie_Anteil",
                  title="Anteilige Entwicklung (aggregiert, 100% Ansicht)",
                  labels={"Prozentualer_Anteil": "Prozentualer Anteil (%)"},
                  color_discrete_map={'Anteil Inländ. Erneuerbare (inkl. Müll)': '#2ca02c', 'Anteil Fossile Importe': '#ff7f0e'},
                  groupnorm='percent')
    fig.update_layout(title_x=0.5, legend_title_text='Energiekategorie', hovermode="x unified")
    fig.update_yaxes(ticksuffix="%")
    return fig

In [12]:
def plot_f3_s2_verhaeltnis_tab():
    if df.empty: return go.Figure().update_layout(title_text="Daten nicht geladen")
    fig = px.line(df_vergleich_s2_f3, x="Jahr", y="Verhaeltnis_ErnIn_vs_FossImp",
                  title="Verhältnis Inl. Erneuerbare (inkl. Müll) zu Fossilen Importen",
                  labels={"Verhaeltnis_ErnIn_vs_FossImp": "Verhältnis"}, markers=True)
    fig.add_hline(y=1, line_dash="dash", line_color="grey", annotation_text="Gleichstand")
    fig.update_layout(title_x=0.5, hovermode="x unified")
    return fig

In [13]:
def plot_f3_s1_anteil_summe_tab():
    if df_vergleich_s1_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 1 nicht geladen")

    fig = px.line(df_vergleich_s1_f3,
                  x='Jahr',
                  y='Anteil_Erneuerbare_Inland_Prozent (Basis)',
                  title='Anteil Inl. Erneuerbare (Basis) an (Inl. Erneuerbare (Basis) + Fossile Importe)',
                  labels={'Anteil_Erneuerbare_Inland_Prozent (Basis)': 'Anteil (%)'},
                  markers=True,
                  line_shape='linear') # Sorgt für eine durchgehende Linie

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Anteil (%)",
        hovermode="x unified",
        yaxis_ticksuffix="%" # Fügt das %-Zeichen zur Y-Achse hinzu
    )
    return fig

In [14]:
def plot_f3_s2_anteil_summe_tab():
    if df_vergleich_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen")

    fig = px.line(df_vergleich_s2_f3,
                  x='Jahr',
                  y='Anteil_Erneuerbare_Inland_Prozent (inkl. Müll)',
                  title='Anteil Inl. Erneuerbare (inkl. Müll) an (Inl. Erneuerbare (inkl. Müll) + Fossile Importe)',
                  labels={'Anteil_Erneuerbare_Inland_Prozent (inkl. Müll)': 'Anteil (%)'},
                  markers=True,
                  line_shape='linear',
                  color_discrete_sequence=['#2ca02c']) # Passende Farbe für "inkl. Müll"

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Anteil (%)",
        hovermode="x unified",
        yaxis_ticksuffix="%"
    )
    return fig

In [15]:
def plot_f3_s1_anteil_100prozent_detail_tab():
    if df_detail_s1_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 1 (Basis) nicht geladen")

    # Um sicherzustellen, dass die Reihenfolge im Stack und in der Legende konsistent ist
    # und die Hauptkategorien (Erneuerbar vs. Fossil) gruppiert bleiben:
    df_detail_s1_f3_sorted = df_detail_s1_f3.copy()
    # Erstelle eine benutzerdefinierte Sortierreihenfolge
    # Erst alle erneuerbaren, dann alle fossilen
    kategorie_reihenfolge_detail_s1 = erneuerbare_inland_basis_f3 + fossile_import_traeger_f3
    df_detail_s1_f3_sorted['Energietraeger'] = pd.Categorical(
        df_detail_s1_f3_sorted['Energietraeger'],
        categories=kategorie_reihenfolge_detail_s1,
        ordered=True
    )
    df_detail_s1_f3_sorted = df_detail_s1_f3_sorted.sort_values(by=['Jahr', 'Energietraeger'])


    fig = px.area(df_detail_s1_f3_sorted,
                  x="Jahr",
                  y="TJ",
                  color="Energietraeger",
                  line_group="Energietraeger", # Wichtig für korrekte Stapelung bei px.area
                  groupnorm='percent', # Normalisiert die Y-Werte auf 100% pro X-Wert
                  title="Anteilige Entwicklung (detailliert, 100% Ansicht): Erneuerbare (Basis) vs. Fossile Importe",
                  labels={"TJ": "Prozentualer Anteil (%)", "Energietraeger": "Energieträger"},
                  color_discrete_map=color_map_f3_szenario1_detail,
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s1} # Legt Reihenfolge in Legende fest
                 )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        yaxis_ticksuffix="%" # Fügt das %-Zeichen zur Y-Achse hinzu
    )
    return fig

In [16]:
def plot_f3_s2_anteil_100prozent_detail_tab():
    if df_detail_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 (inkl. Müll) nicht geladen")

    df_detail_s2_f3_sorted = df_detail_s2_f3.copy()
    kategorie_reihenfolge_detail_s2 = erneuerbare_inland_mit_muell_f3 + fossile_import_traeger_f3
    df_detail_s2_f3_sorted['Energietraeger'] = pd.Categorical(
        df_detail_s2_f3_sorted['Energietraeger'],
        categories=kategorie_reihenfolge_detail_s2,
        ordered=True
    )
    df_detail_s2_f3_sorted = df_detail_s2_f3_sorted.sort_values(by=['Jahr', 'Energietraeger'])

    fig = px.area(df_detail_s2_f3_sorted,
                  x="Jahr",
                  y="TJ",
                  color="Energietraeger",
                  line_group="Energietraeger",
                  groupnorm='percent',
                  title="Anteilige Entwicklung (detailliert, 100% Ansicht): Erneuerbare (inkl. Müll) vs. Fossile Importe",
                  labels={"TJ": "Prozentualer Anteil (%)", "Energietraeger": "Energieträger"},
                  color_discrete_map=color_map_f3_szenario2_detail,
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s2}
                 )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        yaxis_ticksuffix="%"
    )
    return fig

In [17]:
# --- Module installieren und Panel-Dashboard erstellen ---
dashboard_f3 = pn.Tabs(
    ("G01", pn.pane.Plotly(plot_f3_s1_absolut_aggregiert_tab())),
    ("G02", pn.pane.Plotly(plot_f3_s1_absolut_detail_tab())),
    ("G03", pn.pane.Plotly(plot_f3_s1_anteil_100prozent_aggregiert_tab())),
    ("G04", pn.pane.Plotly(plot_f3_s1_verhaeltnis_tab())),
    ("G05", pn.pane.Plotly(plot_f3_s2_absolut_aggregiert_tab())),
    ("G06", pn.pane.Plotly(plot_f3_s2_absolut_detail_tab())),
    ("G07", pn.pane.Plotly(plot_f3_s2_anteil_100prozent_aggregiert_tab())),
    ("G08", pn.pane.Plotly(plot_f3_s2_verhaeltnis_tab())),
    ("G09", pn.pane.Plotly(plot_f3_s1_anteil_summe_tab())), # NEU für Szenario 1
    ("G10", pn.pane.Plotly(plot_f3_s2_anteil_summe_tab())), # NEU für Szenario 2
    ("G11", pn.pane.Plotly(plot_f3_s1_anteil_100prozent_detail_tab())), # NEU
    ("G12", pn.pane.Plotly(plot_f3_s2_anteil_100prozent_detail_tab())), # NEU


    dynamic=True # Lädt Tabs bei Bedarf, kann bei vielen Tabs performanter sein
)

# --- Anzeige des Dashboards ---
dashboard_f3.servable(title="F3: Analyse Erneuerbare Inland vs. Fossile Importe")

Tabs(dynamic=True, sizing_mode='stretch_width')
    [0] Plotly(Figure, sizing_mode='stretch_width')
    [1] Plotly(Figure, sizing_mode='stretch_width')
    [2] Plotly(Figure, sizing_mode='stretch_width')
    [3] Plotly(Figure, sizing_mode='stretch_width')
    [4] Plotly(Figure, sizing_mode='stretch_width')
    [5] Plotly(Figure, sizing_mode='stretch_width')
    [6] Plotly(Figure, sizing_mode='stretch_width')
    [7] Plotly(Figure, sizing_mode='stretch_width')
    [8] Plotly(Figure, sizing_mode='stretch_width')
    [9] Plotly(Figure, sizing_mode='stretch_width')
    [10] Plotly(Figure, sizing_mode='stretch_width')
    [11] Plotly(Figure, sizing_mode='stretch_width')